<a href="https://colab.research.google.com/github/JohnTan38/LangChain-and-LLM/blob/master/YT_Chatting_with_Flan20B_UL2_using_LangChain_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install huggingface_hub langchain transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 1.4 MB/s eta 0:00:00


In [3]:
!pip show langchain

Name: langchain
Version: 0.0.123
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: aiohttp, dataclasses-json, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [4]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [5]:
from langchain.llms import HuggingFaceHub

In [6]:
flan_ul2 = HuggingFaceHub(
    repo_id="google/flan-ul2", 
    model_kwargs={"temperature":0.1,
                  "max_new_tokens":256})

In [7]:
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":0 }
)

In [8]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [9]:
memory = ConversationBufferMemory()

In [10]:
conversation = ConversationChain(
    llm=flan_ul2, 
    verbose=True, 
    memory=memory
)

In [11]:
conversation.predict(input="Hi there! I am John")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am John
AI:

> Finished chain.


'Hi John, how can I help you?'

In [12]:
 conversation.predict(input="How are you today?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am John
AI: Hi John, how can I help you?
Human: How are you today?
AI:

> Finished chain.


'I am good, thanks.'

In [13]:
conversation.predict(input=" Can you help me with some customer support?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am John
AI: Hi John, how can I help you?
Human: How are you today?
AI: I am good, thanks.
Human:  Can you help me with some customer support?
AI:

> Finished chain.


'I can help you with that.'

In [14]:
conversation.predict(input="My TV is broken. can you help fix it?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am John
AI: Hi John, how can I help you?
Human: How are you today?
AI: I am good, thanks.
Human:  Can you help me with some customer support?
AI: I can help you with that.
Human: My TV is broken. can you help fix it?
AI:

> Finished chain.


'Yes, I can help you with that.'

## Counting the tokens

In [15]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

In [16]:
test_input = "Now Good Morning Ms Rogers"
# tokenizer([test_input])
tokenizer.tokenize(test_input) 

['▁Now', '▁Good', '▁Morning', '▁M', 's', '▁Roger', 's']

In [17]:
conversation.memory.buffer

'Human: Hi there! I am John\nAI: Hi John, how can I help you?\nHuman: How are you today?\nAI: I am good, thanks.\nHuman:  Can you help me with some customer support?\nAI: I can help you with that.\nHuman: My TV is broken. can you help fix it?\nAI: Yes, I can help you with that.'

In [18]:
formatted_prompt = conversation.prompt.format(input='the next input',history=memory.buffer)
formatted_prompt

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: Hi there! I am John\nAI: Hi John, how can I help you?\nHuman: How are you today?\nAI: I am good, thanks.\nHuman:  Can you help me with some customer support?\nAI: I can help you with that.\nHuman: My TV is broken. can you help fix it?\nAI: Yes, I can help you with that.\nHuman: the next input\nAI:'

In [19]:
def chat_to_llm(chat_llm ):
    conversation_total_tokens = 0
    new_conversation = ConversationChain(llm=chat_llm, 
                                     verbose=False, 
                                     memory=ConversationBufferMemory()
                                     )
    
    while True:
        message = input("Human: ")
        if message=='exit':
            print(f"{conversation_total_tokens} tokens used in total in this conversation")
            break
        if message:
            formatted_prompt = conversation.prompt.format(input=message,history=new_conversation.memory.buffer)
            num_tokens = len(tokenizer.tokenize(formatted_prompt))
            conversation_total_tokens += num_tokens
            print(f'tokens sent {num_tokens}')
            response = new_conversation.predict(input=message)
            response_num_tokens = len(tokenizer.tokenize(response))
            conversation_total_tokens += response_num_tokens
            print(f"LLM: {response}")


    

In [20]:
chat_to_llm(flan_ul2)

Human: Hi how are you?
tokens sent 64
LLM: I am fine.
Human: How's the weather today?
tokens sent 79
LLM: It is sunny.
Human: exit
151 tokens used in total in this conversation


In [ ]:
chat_to_llm(flan_t5)


Human: hi how are you?
tokens sent 64
LLM: I am good thanks.
Human: What is your name?
tokens sent 78
LLM: I am a robot.
Human: exit
153 tokens used in total in this conversation


## Using a Summary Memory

In [21]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

In [22]:
summary_memory = ConversationSummaryMemory(llm=flan_ul2)

conversation = ConversationChain(
    llm=flan_ul2, 
    verbose=True, 
    memory=summary_memory
)

In [ ]:
conversation.predict(input="Hi there! I am John")

In [ ]:
conversation.predict(input="How are you today?")

In [ ]:
conversation.predict(input="I am good. What is your name?")

In [ ]:
conversation.predict(input="Thats cool can you help me with customer support?")

In [ ]:
conversation.predict(input="What time is your shop open today?")